## Load images

In [1]:
import glob
import matplotlib.image as mpimg

def load_images(pattern):
    pathnames = glob.glob(pattern)
    return [mpimg.imread(x) for x in pathnames]

vehicles = load_images('data/vehicles/*/*.png')
non_vehicles = load_images('data/non-vehicles/*/*.png')

print('Loaded {} vehicle and {} non-vehicle images'.format(len(vehicles), len(non_vehicles)))

Loaded 8792 vehicle and 8968 non-vehicle images


## Extract features

In [2]:
import cv2
import numpy as np
from skimage.feature import hog

def extract_features(image, hog_orientations, hog_px_per_cell, hog_cells_per_block):
    """Extracts an unnormalized feature vectors from provided image"""
    
    grayscale = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    
    hog_features = hog(grayscale, hog_orientations, 
                       (hog_px_per_cell, hog_px_per_cell),
                       (hog_cells_per_block, hog_cells_per_block))
    
    return hog_features

# Feature extraction parameters
HOG_ORIENTATIONS = 9
HOG_PX_PER_CELL = 16
HOG_CELLS_PER_BLOCK = 2

vehicle_features = np.vstack([
    extract_features(image, HOG_ORIENTATIONS, HOG_PX_PER_CELL, HOG_CELLS_PER_BLOCK)
    for image in vehicles
])

non_vehicle_features = np.vstack([
    extract_features(image, HOG_ORIENTATIONS, HOG_PX_PER_CELL, HOG_CELLS_PER_BLOCK)
    for image in non_vehicles
])

print('Number of features: {}'.format(vehicle_features.shape[1]))

/home/goldob/miniconda3/envs/carnd-term1/lib/python3.5/site-packages/skimage/feature/_hog.py:119: skimage_deprecation: Default value of `block_norm`==`L1` is deprecated and will be changed to `L2-Hys` in v0.15
  'be changed to `L2-Hys` in v0.15', skimage_deprecation)


Number of features: 324


## Split data into training & test sets

In [3]:
from sklearn.model_selection import train_test_split

# Use constant seed so that the train-test split remains the same between consecutive executions
SEED = 533343803

X = np.vstack([vehicle_features, non_vehicle_features])
y = np.hstack([np.ones(len(vehicles)), np.zeros(len(non_vehicles))])

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=SEED)

## Normalize features

In [4]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler().fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)